In [ ]:
import os

os.environ["CAPSTONE_OPENAI_API"] = ""

In [8]:
import os
os.chdir(r'c:\Users\ska69\capstone_project')
print(os.getcwd()) 


c:\Users\ska69\capstone_project


In [25]:
from agents.technical_agent import TechnicalAgent
import torch, json, os

# 1. 에이전트 생성
agent = TechnicalAgent(ticker="NVDA")


In [26]:
agent

TechnicalAgent(
  (lstm1): LSTM(13, 64, batch_first=True)
  (lstm2): LSTM(64, 32, batch_first=True)
  (fc): Linear(in_features=32, out_features=1, bias=True)
  (drop): Dropout(p=0.18778570103014075, inplace=False)
  (loss_fn): HuberLoss()
)

In [27]:
# 2. 데이터 수집 및 지표 계산(5개년) → yfinance에서 5년 데이터 다시 받음 + 13개 TECH 지표 생성
X = agent.searcher(rebuild=True)

⚙️ NVDA TechnicalAgent dataset not found. Building new dataset...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


[TechnicalAgent] X_seq: (981, 55, 13), y_seq: (981, 1)
✅ NVDA TechnicalAgent dataset saved to CSV (981 samples, 13 features)
[FundamentalAgent] X_seq: (1221, 14, 13), y_seq: (1221, 1)
✅ NVDA FundamentalAgent dataset saved to CSV (1221 samples, 13 features)
[SentimentalAgent] X_seq: (1221, 14, 8), y_seq: (1221, 1)
✅ NVDA SentimentalAgent dataset saved to CSV (1221 samples, 8 features)


c:\Users\ska69\capstone_project\agents\base_agent.py:240: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period="5y", interval="1d") # 아연수정
[*********************100%***********************]  1 of 1 completed


■ TechnicalAgent StockData 생성 완료 (NVDA, USD)


In [12]:
X.shape

torch.Size([1, 55, 13])

In [28]:
# 3. 모델 학습
# 5년 데이터 기반 LSTM 학습 + 스케일러 저장 + 가중치 저장
agent.pretrain()

[19:30:09] Pretraining TechnicalAgent
  Epoch 005 | Loss: 0.387790
  Epoch 010 | Loss: 0.392585
  Epoch 015 | Loss: 0.393944
  Epoch 020 | Loss: 0.383728
  Epoch 025 | Loss: 0.387176
  Epoch 030 | Loss: 0.393128
  Epoch 035 | Loss: 0.390903
  Epoch 040 | Loss: 0.394917
  Epoch 045 | Loss: 0.384057
 TechnicalAgent 모델 학습 및 저장 완료: models\NVDA_TechnicalAgent.pt


In [29]:
# 4. 최신 윈도우 확보 → 예측 → 설명 생성 → 저장
from config.agents import dir_info

# 최신 윈도우
X_last = agent.searcher(agent.ticker)      # (1,T,F)
dates  = getattr(agent.stockdata, f"{agent.agent_id}_dates", [])


c:\Users\ska69\capstone_project\agents\base_agent.py:240: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period="5y", interval="1d") # 아연수정
[*********************100%***********************]  1 of 1 completed


■ TechnicalAgent StockData 생성 완료 (NVDA, USD)


In [30]:
print(X_last.shape)
print(dates[:5])

torch.Size([1, 55, 13])
['2025-08-07', '2025-08-08', '2025-08-11', '2025-08-12', '2025-08-13']


In [31]:
from agents.base_agent import Target
print(Target.__annotations__)


{'next_close': 'float', 'uncertainty': 'float | None', 'confidence': 'float | None', 'idea': 'dict | None'}


In [32]:
# 5) 예측 + 불확실성
tgt = agent.predict(X_last)            # Target(next_close, uncertainty, confidence, idea)

tgt

Target(next_close=192.0120, uncertainty=0.0044, confidence=0.9956, idea={'per_time': [{'date': '2025-08-07', 'sum_abs': 0.0623}, {'date': '2025-08-08', 'sum_abs': 0.0503}, {'date': '2025-08-11', 'sum_abs': 0.0421}, {'date': '2025-08-12', 'sum_abs': 0.027}, {'date': '2025-08-13', 'sum_abs': 0.0256}, {'date': '2025-08-14', 'sum_abs': 0.0235}, {'date': '2025-08-15', 'sum_abs': 0.0233}, {'date': '2025-08-18', 'sum_abs': 0.0225}, {'date': '2025-08-19', 'sum_abs': 0.0268}, {'date': '2025-08-20', 'sum_abs': 0.0232}, {'date': '2025-08-21', 'sum_abs': 0.0217}, {'date': '2025-08-22', 'sum_abs': 0.0246}, {'date': '2025-08-25', 'sum_abs': 0.0236}, {'date': '2025-08-26', 'sum_abs': 0.0243}, {'date': '2025-08-27', 'sum_abs': 0.0223}, {'date': '2025-08-28', 'sum_abs': 0.0222}, {'date': '2025-08-29', 'sum_abs': 0.0253}, {'date': '2025-09-02', 'sum_abs': 0.0213}, {'date': '2025-09-03', 'sum_abs': 0.0221}, {'date': '2025-09-04', 'sum_abs': 0.0224}, {'date': '2025-09-05', 'sum_abs': 0.0225}, {'date': '20

In [33]:
# 6) 저장
from agents.base_agent import save_explain_json, load_explain_json

save_explain_json(agent.ticker, agent.agent_id, tgt)

'models/explain\\NVDA_TechnicalAgent_explain.json'

In [34]:
# 6. LLM 메시지 구성 (system, user)
sys_text, user_text = agent._build_messages_opinion(agent.stockdata, tgt)
print(sys_text[:400], "\n---\n", user_text[:400])

c:\Users\ska69\capstone_project\agents\base_agent.py:240: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period="5y", interval="1d") # 아연수정
[*********************100%***********************]  1 of 1 completed


■ TechnicalAgent StockData 생성 완료 (NVDA, USD)
너는 '기술적 분석 전문가'로서, 입력된 기술적 지표(모멘텀, RSI, 거래량 등)와 모델의 예측값(next_close)을 기반으로 이 예측이 타당한 이유(reason)를 한국어로 4~5문장으로 작성하라. 예측값을 재계산하지 말고, 반드시 'reasoning'만 제공하라. 각 문장에 최소 1개의 수치를 포함하여 수치 기반으로 의견을 제시하고, 예측값에 대한 불확실성(σ)과 신뢰도(β)가 크면 신중하게, 신뢰도가 높으면 확신 있게 판단하라. 상위 피처의 가중치도 언급하여, 어떤 피처들이 예측에 중요한 역할을 했는지 설명하라. 출력은 JSON 객체 {"reason": string}만 허용한다. 
---
 아래는 기술적 분석 데이터와 예측 정보이다:
{"ticker": "NVDA", "last_price": 191.49, "next_close": 192.012, "uncertainty": 0.0044, "confidence": 0.9956, "delta_pct": 0.2726, "sigma": 0.0044, "beta": 0.9956, "window_size": 55, "idea": {"per_time": [{"date": "2025-08-07", "sum_abs": 0.0623}, {"date": "2025-08-08", "sum_abs": 0.0503}, {"date": "2025-08-11", "sum_abs": 0.0421}, {"date": "2025-08-12", "sum_abs": 0.027}, 


In [35]:
# 7. LLM 호출 및 응답 확인
op = agent.reviewer_draft(agent.stockdata, tgt)
op

c:\Users\ska69\capstone_project\agents\base_agent.py:240: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period="5y", interval="1d") # 아연수정
[*********************100%***********************]  1 of 1 completed


■ TechnicalAgent StockData 생성 완료 (NVDA, USD)


Opinion(agent_id='TechnicalAgent', target=Target(next_close=192.0120, uncertainty=0.0044, confidence=0.9956, idea={'per_time': [{'date': '2025-08-07', 'sum_abs': 0.0623}, {'date': '2025-08-08', 'sum_abs': 0.0503}, {'date': '2025-08-11', 'sum_abs': 0.0421}, {'date': '2025-08-12', 'sum_abs': 0.027}, {'date': '2025-08-13', 'sum_abs': 0.0256}, {'date': '2025-08-14', 'sum_abs': 0.0235}, {'date': '2025-08-15', 'sum_abs': 0.0233}, {'date': '2025-08-18', 'sum_abs': 0.0225}, {'date': '2025-08-19', 'sum_abs': 0.0268}, {'date': '2025-08-20', 'sum_abs': 0.0232}, {'date': '2025-08-21', 'sum_abs': 0.0217}, {'date': '2025-08-22', 'sum_abs': 0.0246}, {'date': '2025-08-25', 'sum_abs': 0.0236}, {'date': '2025-08-26', 'sum_abs': 0.0243}, {'date': '2025-08-27', 'sum_abs': 0.0223}, {'date': '2025-08-28', 'sum_abs': 0.0222}, {'date': '2025-08-29', 'sum_abs': 0.0253}, {'date': '2025-09-02', 'sum_abs': 0.0213}, {'date': '2025-09-03', 'sum_abs': 0.0221}, {'date': '2025-09-04', 'sum_abs': 0.0224}, {'date': '202